<a href="https://www.kaggle.com/code/shaikfardeen1/twitter-hate-py?scriptVersionId=154982476" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('/kaggle/input/twitter-hatespeech/train.csv')
print("Training Set:"% train.columns, train.shape, len(train))

Training Set: (31962, 3) 31962


In [4]:
test = pd.read_csv('/kaggle/input/twitter-hatespeech/test.csv')
print("Test Set:"% test.columns, test.shape, len(test))

Test Set: (17197, 2) 17197


In [5]:
print("Training Set:", train.columns, train.shape, len(train))
print("Test Set:", test.columns, test.shape, len(test))

Training Set: Index(['id', 'label', 'tweet'], dtype='object') (31962, 3) 31962
Test Set: Index(['id', 'tweet'], dtype='object') (17197, 2) 17197


In [6]:
#Data Cleaning
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

In [7]:
#Handling Imbalanced data for Hate Speech Detection Model
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'], train_upsampled['label'], random_state=0)


In [10]:
#Now let’s train the model and predict the results on the test set using the F1 score method:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9694707372350352

In [11]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline_sgd, X_train, y_train, cv=5, scoring='f1')
print("Cross-validated F1 scores:", scores)
print("Mean F1 score:", scores.mean())

Cross-validated F1 scores: [0.96746538 0.96310935 0.96634879 0.96234493 0.96450159]
Mean F1 score: 0.9647540094718288


In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

def preprocess_input(text):
    text = text.lower()
    text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    return text

pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])


new_input_text = input("Enter a text to check for hate speech: ")
preprocessed_input = preprocess_input(new_input_text)
input_features = pipeline_sgd.named_steps['vect'].transform([preprocessed_input])

